In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import os
import detect
#import tflite as tf
import platform
import datetime
import cv2
import time
import numpy as np
import io
from io import BytesIO
from flask import Flask, request, Response, jsonify
import random
import re
import base64
import matplotlib.pyplot as plt
import json

In [6]:
folder_path = r'C:\Users\maxsc\Desktop\TU\DIC3\Pictures_small'
images_s = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)

    # Check if the file is a JPEG image
    if os.path.isfile(file_path) and filename.lower().endswith(".jpg"):
        with open(file_path, "rb") as file:
            # Read the image data
            image_data = file.read()

            # Encode the image data as base64
            encoded_data = base64.b64encode(image_data).decode("utf-8")

            # Append the encoded string to the list
            images_s.append(encoded_data)


In [7]:
#Preprocess ONE image
imgs = []

for i in images_s:
    image_bytes = base64.b64decode(i)
    image = Image.open(io.BytesIO(image_bytes))
    image = image.resize((300, 300))  # Resize to match the model's input size
    image_np = np.array(image).astype(np.float32)  # Convert PIL image to NumPy array
    imgs.append(image_np)

296

In [51]:
images = imgs[:5]


In [3]:
#Load detector model

model_path = r"https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"
detector = hub.load(model_path).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [44]:
def add_detections(result,det,max_length=10):

    #byte to string because byte does not work with json
    aux_ent = [byte.decode() for byte in result["detection_class_entities"][:max_length].tolist()]

    #add results to global detections
    det["entities"].extend(aux_ent)
    det["scores"].extend(result["detection_scores"][:max_length].tolist())
    det["boxes"].extend(result["detection_boxes"][:max_length].tolist())
    

In [48]:
def detections_loop(images,max_length):
    #initialize global detections
    det = {"entities" : [], "scores" : [], "boxes" : []}

    for i in images:
        #converted_img  = tf.image.convert_image_dtype(i, tf.float32)[tf.newaxis, ...]    WORKS TOO
        converted_image = tf.convert_to_tensor(i,tf.float32)[tf.newaxis, ...]
        result = detector(converted_image)
        result = {key:value.numpy() for key,value in result.items()}

        add_detections(result,det,max_length = max_length)

    # Convert dictionary to JSON string
    json_data = json.dumps(det)
    return json_data


In [50]:
detections_loop(images, max_length=20)


{'entities': ['Person',
  'Poster',
  'Human face',
  'Car',
  'Person',
  'Window',
  'Clothing',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Clothing',
  'Clothing',
  'Window',
  'Clothing',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Street light',
  'Window',
  'Window',
  'Person',
  'Person',
  'Person',
  'Tree',
  'Person',
  'Person',
  'Car',
  'Person',
  'Window',
  'Car',
  'Building',
  'Street light',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window blind',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Window',
  'Street light',
  'Street light',
  'Street light',
  'Street light',
  'Window',
  'Window',
  'Window',
  'Vehicle',
  'Window',
  'Street light',
  'Window',
  'Street light',
  'Window',
  'Street light',
  'Window',
  'Window',
  'Street li